In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/data.csv')
#data = pd.DataFrame.from_csv('data/data.csv')

## Get Id's where Age is 0 

In [2]:
zeroAge = data[data['Age'] == 0].index.tolist()

In [3]:
runnerId = [data.iloc[x]['Id'] for x in zeroAge]

In [4]:
for n in runnerId:
    print data[data['Id'] == n]

      Id  Age  Rank  Year   Time  Pace  Sex
488  308   45  1204  2016  15137   578    0
489  308    0  1357  2014  15975   609    0
        Id  Age  Rank  Year   Time  Pace  Sex
2041  1229   48  2717  2009  22803   870    1
2042  1229   47  2414  2008  20344   776    1
2043  1229   53  3194  2014  21215   809    1
2044  1229   46  2054  2007  19896   759    1
2045  1229   51  3870  2012  23302   889    1
2046  1229    0  3332  2011  20990   801    1
2047  1229   50  2208  2006  21090   804    1
2048  1229   50  3482  2011  22896   873    1
2049  1229   49  2811  2010  23191   885    1
2050  1229   44  1566  2005  19757   754    1
2051  1229   55  2783  2016  20941   800    1
2052  1229   50  2748  2015  25435   970    1
         Id  Age  Rank  Year   Time  Pace  Sex
10743  6491   43   217  2006  12496   476    1
10744  6491   50   396  2015  13172   502    1
10745  6491    0   264  2012  12452   475    1
10746  6491   42    84  2005  11596   442    1
         Id  Age  Rank  Year   Time

The following runner id's represent instances where the age is unknown, but the id only appears once, therefore age cannot be corrected: [16673,16954,17902,18202,22942]. To deal with these, the unknown age of the id was set to the average age of all runners who ranked within a range of +/- 50 of the runner in that particular year.

In [5]:
def correctSingleInstanceAge(runnerId,data):
    rank = int(data[data['Id'] == runnerId]['Rank'])
    year = int(data[data['Id'] == runnerId]['Year'])
    rangeOfRunners = data[(data['Rank']>rank-50) & (data['Rank']<rank+50)& (data['Year'] == year)]
    avgAge = rangeOfRunners['Age'].sum()/rangeOfRunners['Age'].size

    index = data[data['Id']==runnerId].index
    data.set_value(index, 'Age', avgAge)
    return


In [6]:
ageCorrectionRunnerIds = [16673,16954,17902,18202,22942]
for n in ageCorrectionRunnerIds:
    correctSingleInstanceAge(n,data)

Runner id's [23714,26270] represent unknown identities, despite clearly being different people that ran in the same year. This represents 6 people in 2013 and 4 people in 2014.

In [7]:
zeroAge = data[(data['Age'] == 0)&(data['Id'] != 26270) &(data['Id'] != 23714) ].index.tolist()
runnerId = [data.iloc[x]['Id'] for x in zeroAge]
for i in runnerId:
    print data[data['Id'] == i].sort_values(by='Year')

      Id  Age  Rank  Year   Time  Pace  Sex
489  308    0  1357  2014  15975   609    0
488  308   45  1204  2016  15137   578    0
        Id  Age  Rank  Year   Time  Pace  Sex
2050  1229   44  1566  2005  19757   754    1
2047  1229   50  2208  2006  21090   804    1
2044  1229   46  2054  2007  19896   759    1
2042  1229   47  2414  2008  20344   776    1
2041  1229   48  2717  2009  22803   870    1
2049  1229   49  2811  2010  23191   885    1
2046  1229    0  3332  2011  20990   801    1
2048  1229   50  3482  2011  22896   873    1
2045  1229   51  3870  2012  23302   889    1
2043  1229   53  3194  2014  21215   809    1
2052  1229   50  2748  2015  25435   970    1
2051  1229   55  2783  2016  20941   800    1
         Id  Age  Rank  Year   Time  Pace  Sex
10746  6491   42    84  2005  11596   442    1
10743  6491   43   217  2006  12496   476    1
10745  6491    0   264  2012  12452   475    1
10744  6491   50   396  2015  13172   502    1
         Id  Age  Rank  Year   Time

For the remaining runner Ids, the participant ran in at least one other year where the age information is recorded. By taking the difference in the years, this offset is added to the recorded age.

In [8]:
def offsetAge(idRecorded,idError,data):
    ageRecorded = data.loc[idRecorded]['Age']
    yearRecorded = data.loc[idRecorded]['Year']
    yearError = data.loc[idError]['Year']
    offset = yearError - yearRecorded
    data.set_value(idError, 'Age', ageRecorded+offset)
    return

In [9]:
offsetAge(488,489,data)
offsetAge(2049,2046,data)
offsetAge(10743,10745,data)
offsetAge(13066,13065,data)
offsetAge(24232,24231,data)

In [10]:
data[(data['Id'] != 23714) & (data['Id'] != 26270) & (data['Age']==0)]

,Id,Age,Rank,Year,Time,Pace,Sex


## Smallest Age

In [11]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

9

In [12]:
data[data['Age']==9]

,Id,Age,Rank,Year,Time,Pace,Sex
31786,23689,9,2339,2013,6994,533,1


In [13]:
data[data['Id']==23689]

,Id,Age,Rank,Year,Time,Pace,Sex
31786,23689,9,2339,2013,6994,533,1
31787,23689,45,2339,2013,6994,533,1


Participants ran in the same year with the same id/time/pace/sex. Id 31786 can be perceived as an error and eliminated.

In [14]:
data = data.drop([31786])

In [15]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

10

## Force Ages to increments of 5

In [16]:
for i in range (data['Age'].size):
    data.iloc[i]['Age'] = data.iloc[i]['Age'] - data.iloc[i]['Age']%5

In [17]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

10

In [18]:
#import matplotlib.pyplot as plt
#pd.Series.hist(data['Age'])
#plt.show()

## Add weather data
Temperature in deg. C at 6:53 or 6:59 AM

In [18]:
temp = {
    '2003':10.6,
    '2004':20.6,
    '2005':18.9,
    '2006':21.1,
    '2007':22.2,
    '2008':17.8,
    '2009':14.4,
    '2010':21.7,
    '2011':11.1,
    '2012':20.6,
    '2013':19.4,
    '2014':22.8,
    '2015':11.7,
    '2016':12.2
}

In [19]:
data['Temp'] = 0.

In [20]:
for i in data.index.tolist():
    year = str(int(data.loc[i]['Year']))
    data.set_value(i,'Temp',temp[year])

## Add Number of Races
Number of races run since 2003

In [21]:
data['raceCount'] = 0

In [22]:
for i in data.index.tolist():
    runnerId = data.loc[i]['Id']
    count = int(data[data['Id']==runnerId]['Id'].size)
    data.set_value(i,'raceCount',count)

## Years since Last Race

In [23]:
def yearsSinceLastRace(runnerId,year,data):
    runnerAllYears = data[(data['Id']==runnerId) & (data['Year']<year)].sort_values(by='Year')['Year']
    if( runnerAllYears.size == 0 ):
        lastYear = year
    else:
        lastYear = max(runnerAllYears)
    return year-lastYear
    

In [24]:
for i in data.index.tolist():
    runnerId = data.loc[i]['Id']
    year = data.loc[i]['Year']
    #data.set_value(i,'totalNumRaces',count)
    data.set_value(i,'yrsSinceLast',int(yearsSinceLastRace(runnerId,year,data)))

## 2013 Projections using Riegel's formula
http://cs229.stanford.edu/proj2015/247_report.pdf

In [25]:
def fullMarathonTime(halfMarathonTime,data):
    return np.round(halfMarathonTime*((26.219/13.095)**1.06))

In [26]:
halfMarathonId = data[data['Year']==2013].index.tolist()
data['fullMarathonTime'] = np.NaN

In [27]:
for i in halfMarathonId:
    time = data.loc[i]['Time']
    data.set_value(i,'fullMarathonTime',int(fullMarathonTime(time,data)))

## Normalize data

In [29]:
data

,Id,Age,Rank,Year,Time,Pace,Sex,Temp,raceCount,yrsSinceLast,fullMarathonTime
0,1,30,1263,2016,15300,584,1,12.2,1,0.0,NaN
1,2,60,2289,2016,18342,701,1,12.2,1,0.0,NaN
2,3,30,2276,2016,18274,698,0,12.2,1,0.0,NaN
3,4,35,670,2015,13977,533,1,11.7,2,0.0,NaN
4,4,35,2390,2016,18810,718,1,12.2,2,1.0,NaN
5,5,40,2608,2016,19714,753,0,12.2,9,2.0,NaN
6,5,40,3086,2014,20594,786,0,22.8,9,2.0,NaN
7,5,35,2455,2010,19744,753,0,21.7,9,1.0,NaN
8,5,35,2343,2008,19729,753,0,17.8,9,1.0,NaN
9,5,35,2415,2009,19647,749,0,14.4,9,1.0,NaN


## 